In [ ]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import datetime as dt

In [ ]:
import sqlite3
connection = sqlite3.connect('hawaii.sqlite')


In [ ]:
connection.row_factory = sqlite3.Row 

In [ ]:
# cursor = connection.execute('select * measurement')
# # instead of cursor.description:
# row = cursor.fetchone()
# names = row.keys()

# Reflect Tables into SQLAlchemy ORM

In [ ]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [ ]:
engine = create_engine('sqlite:///hawaii.sqlite')

In [ ]:
# We can view all of the classes that automap found

Base = automap_base()
Base.prepare(engine, reflect=True)


In [ ]:
Base.classes.station

In [ ]:
Base.classes.keys()

In [ ]:
# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.measurement

In [ ]:
from sqlalchemy import inspect
inst = inspect(Measurement)
meas_names = [c_attr.key for c_attr in inst.mapper.column_attrs]
meas_names

In [ ]:
from sqlalchemy import inspect
inst = inspect(Station)
sta_names = [c_attr.key for c_attr in inst.mapper.column_attrs]
sta_names

In [ ]:
# Create our session (link) from Python to the DB
session = Session(engine)


In [ ]:
date=session.query(Measurement.date).all()
#date

In [ ]:
measure = session.query(Station.prcp).all()
# measure

In [ ]:
results=session.query(Measurement.date, Station.prcp,).all()
#use python to loop through sqlalchemy results
for result in results:
    print(result)
#alternate
# results=session.query(Measurement.date, Station.prcp)
# print(results.all())



In [ ]:
#repeat above using pandas - alternate method

In [ ]:
df=pd.DataFrame(results)
df.head()

In [ ]:
 df['date'].dtype

In [ ]:
df['date'] = pd.to_datetime(df['date'])


In [ ]:
least_recent_date = df['date'].min()
least_recent_date

In [ ]:
end_date = df['date'].max()
end_date

# Exploratory Climate Analysis

In [ ]:
from datetime import timedelta
start_date = end_date - timedelta(days=365)



In [ ]:
start_date

In [ ]:
year_df=df[df['date']>=start_date]
year_df

In [ ]:
year_df.set_index('date', inplace=True)
#year_df


In [ ]:
year_df.plot()

In [ ]:
year_df.sort_values(by=['date'], inplace=True, ascending=False)
year_df.head()

In [ ]:
year_df.describe()

In [ ]:
precipitation = []
precipitation = session.query(Measurement.date, Measurement.prcp, Measurement.tobs)
print(precipitation.all())

In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data
print(year_df['prcp'].sum())


In [ ]:
from sqlalchemy import distinct
# How many stations are available in this dataset?
# count distinct "name" values
session.query(func.count(distinct(Station.station))).all()


In [ ]:
#preference for using assigned variable names
unique_stations=session.query(func.count(distinct(Station.station))).all()
print(unique_stations)

In [ ]:
# How many stations are available in this dataset?
#check stations too
sta_info=session.query(Station.id, Station.prcp, Measurement.tobs, Station.station).all()
for s_detail in sta_info:
    print(s_detail)

In [ ]:
#sqlite query for finding station frequency count/dictionary

session.query(Measurement.station, func.count(Measurement.station)).\
group_by(Measurement.station).order_by(func.count(Measurement.station).desc()).all()

In [ ]:
#find station frequency count using pandas instead of sqlalchemy
sta_df=pd.DataFrame(sta_info)
sta_df.head()

In [ ]:
# What are the most active stations?
# pandas:
station_activity=sta_df['station'].value_counts()
print(station_activity)




In [ ]:
# What are the most active stations?

#pure python to do the same as above which is to get a count of stations by creating a dictionary
#-----------------------------------
# variable assigned to list of station checks which contains station names
station_occurrences = list(sta_df['station'])

#print to verify
# print(station_occurrences)

#create empty dictionary to hold results
station_dictionary={}

#set counter to zero
station_count=0

#loop through list of station checks

for station_name in station_occurrences:
    #for each name in the station checks, retain total count to be able to establish percentages later 
    station_count+=1

    # for each station name in the station checks, if the name is not in the dictionary, .....(continued)
    if station_name not in station_dictionary:    
        #(continued) ... then make sure the count of that is equal to zero for the station name. This automatically creates a station name as a key.  

    # This is a "hack", because we are outside of the if statment (forcing function). We set outside the "if" loop, which closes the "if" loop and  appends the name of the station to the dictionary with a value of '0'. 
        station_dictionary[station_name] = 0
    #inside of the "for" loop, we add to the station a count of one.
    station_dictionary[station_name] += 1


    #continue loop till over
#when looping through values ends, dictionary is printed along with station count.
print(station_dictionary)
print(station_count)




In [ ]:
#most active station using python
max_value=max(station_dictionary)

print(max_value)

In [ ]:
top_count=[]
for k,v in station_dictionary.items():
    # v is the list of counts per station
    if v not in top_count:
        top_count.append(v)
    print(k,v)
#print value of highest count for a unique station ('USC00519523')
print(max(top_count))

In [ ]:
top_dict={}
top_count=[]
for k,v in station_dictionary.items():
    # v is the list of counts per station
    if v not in top_count:
        top_count.append(v)

    print(k,v)
    
#print value of highest count for a unique station ('USC00519523')
print(f'The highest count for a unique station is USC00519523: {max(top_count)}')

In [ ]:
# What are the most active stations?

#python method (different strategy)

from collections import Counter

#all stations
print(Counter(station_occurrences))

#most active
max(Counter(station_occurrences))


In [ ]:
# Using the station id from the previous query, calculate the lowest temperature recorded, 
session.query(func.min(Measurement.tobs)).\
filter(Measurement.station == 'USC00519281').all()


In [ ]:
# highest temperature recorded, and average temperature most active station?

session.query(func.min(Measurement.tobs), func.max(Measurement.tobs), func.avg(Measurement.tobs)).\
filter(Measurement.station == 'USC00519281').all()


In [ ]:
highest_count_station= sta_df.groupby("station")["tobs"].max()
highest_count_station

In [ ]:
sta_df[sta_df['station']=='USC00519281']

In [ ]:
# sta_df[sta_df['id']=='USC00519281']
sta=sta_df[sta_df['station']=='USC00519281']
print(sta)

In [ ]:
sta_df.groupby('station').prcp.max()


In [ ]:
sta_df['tobs'] = sta_df['tobs'].astype(float)
tobs=sta_df[sta_df['station']=='USC00519281'].tobs
print(tobs)

In [ ]:

print(f'The minimum, maximum and average temps for station USC00519281 is: {tobs.min()}, {tobs.max()} and {tobs.mean()}')

In [ ]:
#easy way using the describe function:
sta_df[sta_df['station']=='USC00519281'].tobs.describe()

In [ ]:
#the last year of temps from dataset using query and plotted using pandas as a histogram
m_a_temp = session.query(Measurement.station, Measurement.tobs).\
filter(Measurement.station == 'USC00519281', Measurement.date >= '2016-08-23').\
order_by(Measurement.tobs.desc()).all()


In [ ]:
print(m_a_temp)